In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'building-permit-applications-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F16432%2F21688%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240704%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240704T164141Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D79d782826a4674a6566cddd5a99cccf128fd1a93bc9d8de36eef69a009acc0ce874e8742e09c3264bb81f7afc37641c9244307a098519a04ac413f01b36e1bdbcc352f60784aee5badf2f024c7ecf24d87b0684a63f370d86863157b88cb4a13d1ed4edffc7fd461ee22d3416c2f8e97715a15aeb7a9a67643612b3b023b31641c1b606b651e27c1e2527e9f82a78048b98ab32ec7617cf25d9f352e842bebc47589a9de60a9e96258a13bc5b2973b256698ddd95c2481b9d3c157b23099a7ffce5c495d2b830eb6f6452c5268fd7e0c4121f1e9974a9182e481616f312eeea9f7fc2a5aa0b1ca38a332de9f85cc45f88724ad42091cfee86acfa5a371845ad4'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


**This notebook is an exercise in the [Data Cleaning](https://www.kaggle.com/learn/data-cleaning) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/handling-missing-values).**

---


In this exercise, you'll apply what you learned in the **Handling missing values** tutorial.

# Setup

The questions below will give you feedback on your work. Run the following cell to set up the feedback system.

In [ ]:
from learntools.core import binder
binder.bind(globals())
from learntools.data_cleaning.ex1 import *
print("Setup Complete")

# 1) Take a first look at the data

Run the next code cell to load in the libraries and dataset you'll use to complete the exercise.

In [ ]:
# modules we'll use
import pandas as pd
import numpy as np

# read in all our data
sf_permits = pd.read_csv("../input/building-permit-applications-data/Building_Permits.csv")

# set seed for reproducibility
np.random.seed(0)

Use the code cell below to print the first five rows of the `sf_permits` DataFrame.

In [ ]:
# TODO: Your code here!
sf_permits.head(5)

In [ ]:
sf_permits.isnull().sum()

Does the dataset have any missing values?  Once you have an answer, run the code cell below to get credit for your work.

In [ ]:
# Check your answer (Run this code cell to receive credit!)
q1.check()

In [ ]:
# Line below will give you a hint
#q1.hint()

# 2) How many missing data points do we have?

What percentage of the values in the dataset are missing?  Your answer should be a number between 0 and 100.  (If 1/4 of the values in the dataset are missing, the answer is 25.)

In [ ]:
# TODO: Your code here!
total_missing = sf_permits.isnull().sum()

# Calculating total number of values
total_cells = sf_permits.shape[0]
percent_missing = (total_missing/total_cells)*100

# Check your answer
q2.check()

In [ ]:
# Lines below will give you a hint or solution code
#q2.hint()
#q2.solution()

# 3) Figure out why the data is missing

Look at the columns **"Street Number Suffix"** and **"Zipcode"** from the [San Francisco Building Permits dataset](https://www.kaggle.com/aparnashastry/building-permit-applications-data). Both of these contain missing values.
- Which, if either, are missing because they don't exist?
- Which, if either, are missing because they weren't recorded?  

Once you have an answer, run the code cell below.

In [ ]:
# Display the first few rows of the relevant columns

print(sf_permits[['Street Number Suffix', 'Zipcode']])

In [ ]:
# Check the number of missing values in each column
missing_values = sf_permits[['Street Number Suffix', 'Zipcode']].isnull().sum()

print(missing_values)

In [ ]:
# Analyze patterns and domain knowledge
# Validate Assumptions with data analysis
# Analize the street number suffix column

street_number_suffix_unique_values = sf_permits['Street Number Suffix'].unique()
print('Unique Values in street_number_suffix_unique_values: ')
print(street_number_suffix_unique_values)

# Analyze the zipcode column

zipcode_unique_values = sf_permits['Zipcode'].unique()
print("Unique Values in Zipcode: ")
print(zipcode_unique_values )

In [ ]:
# Check your answer (Run this code cell to receive credit!)
q3.check()

In [ ]:
# Line below will give you a hint
#q3.hint()

# 4) Drop missing values: rows

If you removed all of the rows of `sf_permits` with missing values, how many rows are left?

**Note**: Do not change the value of `sf_permits` when checking this.  

In [ ]:
# TODO: Your code here!
sf_permits
# Print original number of rows
original_row_count = sf_permits.shape[0]

print("Original number of rows: ", original_row_count)

sf_permits_cleaned = sf_permits.dropna()

# Print the number of remaining rows

remaining_row_count = sf_permits_cleaned.shape[0]

print("Number of rows after cleaning: ", remaining_row_count)

# Calculate and print number of rows removed

rows_removed = original_row_count - remaining_row_count

print("Number of row removed: ", rows_removed)


Once you have an answer, run the code cell below.

In [ ]:
# Check your answer (Run this code cell to receive credit!)
q4.check()

In [ ]:
# Line below will give you a hint
#q4.hint()

# 5) Drop missing values: columns

Now try removing all the columns with empty values.  
- Create a new DataFrame called `sf_permits_with_na_dropped` that has all of the columns with empty values removed.  
- How many columns were removed from the original `sf_permits` DataFrame? Use this number to set the value of the `dropped_columns` variable below.

In [ ]:
# TODO: Your code here
sf_permits_with_na_dropped = sf_permits.dropna(axis=1)

# print the remaining number of columns

remaining_column_count = sf_permits_with_na_dropped.shape[1]
print("Number of columns after removing columns with missing values: ", remaining_column_count )

dropped_columns = sf_permits_with_na_dropped-remaining_column_count
print("Dropped coulumns: ", dropped_columns)

# Check your answer
q5.check()

In [ ]:
# Lines below will give you a hint or solution code
#q5.hint()
#q5.solution()

# 6) Fill in missing values automatically

Try replacing all the NaN's in the `sf_permits` data with the one that comes directly after it and then replacing any remaining NaN's with 0.  Set the result to a new DataFrame `sf_permits_with_na_imputed`.

In [ ]:
# TODO: Your code here
import pandas as pd
from sklearn.impute import SimpleImputer

sf_permits
# Separate the dataset into numeric and categorical columns
numeric_cols = sf_permits.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = sf_permits.select_dtypes(include=['object']).columns

# Create an imputer for numeric columns (using mean)
numeric_imputer = SimpleImputer(strategy='mean')
sf_permits[numeric_cols] = numeric_imputer.fit_transform(sf_permits[numeric_cols])

# Create an imputer for categorical columns (using most frequent)
categorical_imputer = SimpleImputer(strategy='most_frequent')
sf_permits[categorical_cols] = categorical_imputer.fit_transform(sf_permits[categorical_cols])

# The DataFrame with imputed values
sf_permits_with_na_imputed = sf_permits

# Optionally, save the new DataFrame to a CSV file
sf_permits_with_na_imputed.to_csv('Building_Permits_with_Imputed_Values.csv', index=False)

# Print a summary to verify the changes
print(f"Number of missing values in each column after imputation:\n{sf_permits_with_na_imputed.isnull().sum()}")


# Check your answer
q6.check()

In [ ]:
# Lines below will give you a hint or solution code
#q6.hint()
#q6.solution()

# More practice

If you're looking for more practice handling missing values:

* Check out [this noteboook](https://www.kaggle.com/alexisbcook/missing-values) on handling missing values using scikit-learn's imputer.
* Look back at the "Zipcode" column in the `sf_permits` dataset, which has some missing values. How would you go about figuring out what the actual zipcode of each address should be? (You might try using another dataset. You can search for datasets about San Fransisco on the [Datasets listing](https://www.kaggle.com/datasets).)

# Keep going

In the next lesson, learn how to [**apply scaling and normalization**](https://www.kaggle.com/alexisbcook/scaling-and-normalization) to transform your data.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/data-cleaning/discussion) to chat with other learners.*